In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!pip install -q transformers datasets accelerate scikit-learn sentencepiece

In [18]:
#importing the necessary libraries
import torch
import pandas as pd
import re
import matplotlib.pyplot as plt




In [19]:
cd /content/drive/MyDrive/mbti-bert-master

/content/drive/MyDrive/mbti-bert-master


In [20]:
import torch
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from transformers import BigBirdTokenizer, BigBirdForSequenceClassification, Trainer, TrainingArguments

def clean_posts(df):
    df['posts'] = df['posts'].str.replace(r'\|\|\|', ' ', regex=True)
    df['posts'] = df['posts'].str.replace(r"http\S+", "", regex=True)
    df['posts'] = df['posts'].str.replace(r"[-/@.?!_,:;()|0-9]", "", regex=True)
    df['posts'] = df['posts'].str.replace(r"\s+", " ", regex=True).str.strip()
    return df

def prepare_mbti_labels(df):
    # I/E: Introversion (0) / Extroversion (1)
    df['IE'] = df['type'].apply(lambda x: 0 if x[0] == 'I' else 1)
    # N/S: Intuition (0) / Sensing (1)
    df['NS'] = df['type'].apply(lambda x: 0 if x[1] == 'N' else 1)
    # T/F: Thinking (0) / Feeling (1)
    df['TF'] = df['type'].apply(lambda x: 0 if x[2] == 'T' else 1)
    # J/P: Judging (0) / Perceiving (1)
    df['JP'] = df['type'].apply(lambda x: 0 if x[3] == 'J' else 1)
    return df

# Load dữ liệu
df = pd.read_csv('mbti_1.csv')
df = clean_posts(df)
df = prepare_mbti_labels(df)




In [21]:
import torch

class MBTIDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Chuyển đổi dữ liệu sang tensor
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [22]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BigBirdTokenizer, BigBirdForSequenceClassification, Trainer

# Sử dụng tên model gốc để load tokenizer
model_name = "google/bigbird-roberta-base"
tokenizer = BigBirdTokenizer.from_pretrained(model_name)
MAX_LEN = 3000

# Hàm ánh xạ ngược từ số sang chữ cái MBTI
def reconstruct_mbti(ie, ns, tf, jp):
    mbti = ""
    mbti += 'E' if ie == 1 else 'I'
    mbti += 'S' if ns == 1 else 'N'
    mbti += 'F' if tf == 1 else 'T'
    mbti += 'P' if jp == 1 else 'J'
    return mbti

In [29]:
import os
dimensions = ['IE', 'NS', 'TF', 'JP']
# Sử dụng os.path.abspath để chuyển './model_mbti_IE' thành đường dẫn đầy đủ
model_paths = {
    'IE': os.path.abspath('IE_model'),
    'NS': os.path.abspath('NS_model'),
    'TF': os.path.abspath('TF_model'),
    'JP': os.path.abspath('JP_model')
}

# Chuẩn bị dữ liệu validation (giữ nguyên random_state=42 như trong code training)
train_texts, val_texts, _, _ = train_test_split(
    df['posts'], df['type'], test_size=0.2, random_state=42
)

val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=MAX_LEN)

results = {}
y_true_all = {}

for dim in dimensions:
    print(f"--- Processing Dimension: {dim} ---")

    # Load model
    model = BigBirdForSequenceClassification.from_pretrained(model_paths[dim])
    # Chuẩn bị nhãn thực tế cho chiều này
    y_true = df.loc[val_texts.index, dim].values
    y_true_all[dim] = y_true

    # Predict bằng Trainer (để tận dụng tối ưu hóa của HuggingFace)
    test_dataset = MBTIDataset(val_encodings, list(y_true))
    trainer = Trainer(model=model)

    predictions = trainer.predict(test_dataset)
    y_pred = np.argmax(predictions.predictions, axis=-1)
    results[dim] = y_pred

    # In Classification Report cho từng chiều
    target_names = ['I', 'E'] if dim == 'IE' else \
                   ['N', 'S'] if dim == 'NS' else \
                   ['T', 'F'] if dim == 'TF' else ['J', 'P']

    print(f"Classification Report for {dim}:")
    print(classification_report(y_true, y_pred, target_names=[target_names[0], target_names[1]]))

--- Processing Dimension: IE ---


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ayukhanh2 (ayukhanh2-hanoi-university-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Classification Report for IE:
              precision    recall  f1-score   support

           I       0.90      0.76      0.82      1353
           E       0.45      0.70      0.55       382

    accuracy                           0.74      1735
   macro avg       0.67      0.73      0.68      1735
weighted avg       0.80      0.74      0.76      1735

--- Processing Dimension: NS ---


Classification Report for NS:
              precision    recall  f1-score   support

           N       0.89      0.94      0.92      1489
           S       0.47      0.32      0.38       246

    accuracy                           0.85      1735
   macro avg       0.68      0.63      0.65      1735
weighted avg       0.83      0.85      0.84      1735

--- Processing Dimension: TF ---


Classification Report for TF:
              precision    recall  f1-score   support

           T       0.79      0.68      0.73       798
           F       0.76      0.85      0.80       937

    accuracy                           0.77      1735
   macro avg       0.77      0.76      0.77      1735
weighted avg       0.77      0.77      0.77      1735

--- Processing Dimension: JP ---


Classification Report for JP:
              precision    recall  f1-score   support

           J       0.48      0.62      0.55       669
           P       0.71      0.58      0.64      1066

    accuracy                           0.60      1735
   macro avg       0.60      0.60      0.59      1735
weighted avg       0.62      0.60      0.60      1735



In [30]:
# Ghép các kết quả lại thành chuỗi 4 chữ cái (Ví dụ: INFJ)
y_pred_final = []
y_true_final = []

for i in range(len(val_texts)):
    # Dự đoán
    pred_mbti = reconstruct_mbti(results['IE'][i], results['NS'][i], results['TF'][i], results['JP'][i])
    y_pred_final.append(pred_mbti)

    # Thực tế
    true_mbti = reconstruct_mbti(y_true_all['IE'][i], y_true_all['NS'][i], y_true_all['TF'][i], y_true_all['JP'][i])
    y_true_final.append(true_mbti)

# In Classification Report tổng quát cho 16 nhãn
print("\n" + "="*50)
print("FINAL CLASSIFICATION REPORT FOR 16 MBTI TYPES")
print("="*50)
print(classification_report(y_true_final, y_pred_final))


FINAL CLASSIFICATION REPORT FOR 16 MBTI TYPES
              precision    recall  f1-score   support

        ENFJ       0.09      0.34      0.14        41
        ENFP       0.28      0.34      0.30       125
        ENTJ       0.13      0.32      0.18        44
        ENTP       0.34      0.29      0.31       135
        ESFJ       0.00      0.00      0.00         7
        ESFP       0.09      0.12      0.11         8
        ESTJ       0.05      0.14      0.08         7
        ESTP       0.12      0.13      0.12        15
        INFJ       0.35      0.39      0.37       288
        INFP       0.44      0.41      0.43       370
        INTJ       0.29      0.30      0.29       193
        INTP       0.49      0.29      0.36       293
        ISFJ       0.13      0.04      0.07        45
        ISFP       0.12      0.08      0.09        53
        ISTJ       0.33      0.23      0.27        44
        ISTP       0.18      0.07      0.11        67

    accuracy                     